## Automatic Definition Finder
- Definition Source：Baidu, Zhihu, reciBaike, GengBaike, lxyBaike.

In [1]:
import numpy as np
import pandas as pd
import requests
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup 
from lxml import etree
import csv,codecs
import re
import xlsxwriter
import math
import time


In [11]:
def find_def(word):
    ret = []
    ret.append("梗百科： " + find_gbk('梗百科.csv',word))
    ret.append("流行语百科： " + find_lyx('lxyBaike.csv',word))
    ret.append("热词百科： " +find_reci('reciBaike.csv',word))
    search_term = word + "是什么梗 知乎"
    baidu = []
    zhihu = []
    try:
        zhihu = Get_zhihu(search_term, 1, word)
    except Exception as e:
        print("知乎bug")
        
    try:
        baidu = find_baidu(word)
    except Exception as e:
        print("百度bug")
    ret = ret + baidu + zhihu
    return ret

### Baidu

In [3]:
def find_baidu(word):   
    ret = []
    headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
    }
    url='https://baike.baidu.com/item/'+word      
    try:
        response = requests.get(url,headers=headers)

        #将一段文档传入BeautifulSoup的构造方法,就能得到一个文档的对象, 可以传入一段字符串
        soup = BeautifulSoup(response.text,'lxml')
    except Exception as e:
        print(e)
        
    definition = soup.find(property="og:description")['content']
    s= soup.get_text()
    origin = ""  #词语来源
    development = "" #词语发展
    example = "" #引用示例

    if ((word+"词语来源") in s):
        origin = s.split(word+"词语来源")[1].replace("\n","").split("\xa0")[0].replace("编辑语音","")
    if ((word+"词语发展") in s):
       development =s.split(word+"词语来源")[1].replace("\n","").split("\xa0")[0].replace("编辑语音","")    
    if ((word+"引用示例") in s):
       example =s.split(word+"引用示例")[1].replace("\n","").split("\xa0")[0].replace("编辑语音","")    
        
        
    ret.append(definition)
    ret.append(origin)
    ret.append(development)
    ret.append(example)
    return ret

def get_baidu(word):
    try:
        res = find_baidu(word)
        for i in range(len(res)):
            if (len(res[i]) != 0):
                print(res[i] + "\n")
    except Exception as e:
        print("该词未被百度收录。")

### lxyBaike

In [4]:

def find_lyx(link,word):
    df = pd.read_csv(link)
    ret = "流行语百科未收录该词。"
    if (word in df["词条"].values):
        ret=(df.loc[df['词条'] == word]['摘要'].iloc[0])
    return ret
       

### reciBaike

In [5]:
def find_reci(link,word):
    df = pd.read_csv(link)
    ret = "热词百科未收录该词。"
    if (word in df["word"].values):
        ret=(df.loc[df['word'] == word]['explanation'].iloc[0])
    return ret
       

### gengBaike

In [6]:
def find_gbk(link,word):
    df = pd.read_csv(link, names=['word','def'], index_col=False)
    ret = "梗百科未收录该词。"
    if (word in df["word"].values):
        ret="梗百科解释：" + (df.loc[df['word'] == word]['def'].iloc[0])
    return ret

### Zhihu

In [7]:
def extract_zhihu(url, word):
    ret = []
    headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
    }   
    try:
        response = requests.get(url,headers=headers)


        soup = BeautifulSoup(response.text,'lxml')
    except Exception as e:
        print(e)

        
#     只取中文和标点
#     t= re.findall('[\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\u4e00-\u9fa5]',s)
#     print(''.join(t))       
        
    test = soup.select('span[name]')
    for x in soup.find_all("span"):
        if (word in x.get_text()):   
            ret.append(x.get_text())
    ret = "\n".join(ret)
    return ret



In [8]:



def convert_url(url):
    headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
    }   
    resp = requests.get(url=url,
                        headers=headers,
                        allow_redirects=False, 
                        verify=False
                        )
    return resp.headers['Location']

    
def get_url(wd,num, search_term):
    
    headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
    }   
    info = []
    s = requests.session()
    total_title=[]
    total_url=[]
    total_info=[]

    num=num*10-10
    for i in range(-10, num, 10):
        url = 'https://www.baidu.com/s'
        params = {
            "wd": wd,
            "pn": i,
        }
        r = s.get(url=url, headers=headers, params=params,verify=False)
        print("返回状态码:",r.status_code) 
        soup = BeautifulSoup(r.text, 'lxml')
        for so in soup.select('#content_left .t a'):
            g_url = convert_url(so.get('href'))
            g_title=so.get_text().replace('\n','').strip()
#             print(g_title,g_url)
            total_title +=[g_title]
            total_url +=[g_url]
        time.sleep(1 + (i / 10))
        
        zhihu_num = 0
        for ur in total_url:
#             print(ur)
            if ("zhihu" in ur):
                app = extract_zhihu(ur,search_term)
                info.append(app)
                zhihu_num +=1
        print("Found " + str(zhihu_num) + "zhihu links")

        return info

def Get_zhihu(wd, page, search_term):   
    if __name__ == '__main__':
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0",
            "Host": "www.baidu.com",
        }
        return get_url(wd,1,search_term)


# Come Search！

In [ ]:
if __name__ == '__main__':
    while True:#循环
        w = input("Enter a slang： ")
        print("enter 'quit' to quit loop.")
        if (w == "quit"):
            break;
        ret = find_def(w)
        for i in range(len(ret)):
            print(ret[i])
            print('\n')
        

Enter a slang： 打野
enter 'quit' to quit loop.


/Users/Eric/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


返回状态码: 200
Found 10zhihu links
梗百科： 梗百科未收录该词。


流行语百科： 流行语百科未收录该词。


热词百科： 在moba类(塔防类)游戏，比如英雄联盟或者王者荣耀中，打野是一个玩家位置，常规任务之一就是打野怪，可以通过打野怪得到经验值和金币。


打野，汉语词汇，拼音是dǎ yě，解释是亦称“打野呵”。谓艺人在街头卖艺。&amp;nbsp;方言。广西壮族自治区桂林话，意指“开小差”、“不专心”、“三心二意”。方言。犹言捞外快。











打野就是指打死野生动物。今天给大家介绍一位中国野王。何广位是1909年出生，2004年去世，享年95岁，是一位长寿的传武大师，本来是安徽人，后来流亡到了河南。自幼天生神力，饭量惊人。他14岁的时候拜江湖卖艺的一位名叫王进的大师为师，学习了一套惊奇的拳法。之后由于和几名鬼子激战搏斗，被迫和恩师分道扬镳。之后独立行走江湖的何广位还曾经路见不平一声吼，打死过两名调戏妇女的日本鬼子。要说何广位真是苦命人，他饥不择食，生活窘迫，偶然间三拳两脚打死了一只野狼，后来拿到了集市上去卖，卖了30大洋。之后尝到甜头的何广位开始以打猎为生。期间他总共打死和活捉7只猛虎，9头野牛，230多只金钱豹，800多只野猪和1000多只恶狼，堪称当代活武松。他靠的就是一双拳头，比武松还硬的拳头。何广位身高1米8多，体格极其健壮，力大无穷，还曾经在山西为民除害，徒手击毙一只金钱豹，被当地传为佳话，而且庆功宴上喝了7斤白酒，比武松还要能打，还能喝。要说何广位为何能有如此神勇的表现，笔者认为绝对力量是第一位的，其次就是仗着师父教给他的一套精湛拳法。两者相结合，才造就了何老英雄的辉煌战绩。如果何广位可以和现在的搏击拳手比武，笔者认为他完全可以一拳KO任何一位搏击拳手，毕竟绝对力量太强悍。太极雷雷、咏春大师余昌华、两仪拳大师吕刚，里合腿田野等人虽然也都是传武高手，但是他们缺乏绝对力量，尽管招式奇特，但无法和真正的搏击拳手对抗，往往遭遇被KO的结果。但是何广位则不同，他身材高大，多年的猎户生涯造就他的抗打能力超强，我们可以想象，他击毙或者活捉2000多只猛兽，不可能没有受过伤，如果没有超强的抗打能力，恐怕早就成为老虎的腹中餐了。反正我全是复制粘贴过来的，我看完我也信了，至于你信不信随便吧······中国第一打野~





这